Please run the cell below to ensure that the rest of the report goes as expected!

In [1]:
library(tidyverse)
library(repr)
library(ggplot2)
library(RColorBrewer)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


---

# From Sessions to Predictions: Modeling Engagement with KNN Regression

---

## Introduction

From the title you're probably wondering, what are 'sessions,' what is being predicted, and most likely what even is the term 'KNN Regression.' Well no need to worry! Our project stems from a UBC Computer Science study called **PLAIcraft** with the goal and I quote, "PLAICraft is a research data collection project with the aim of enabling advanced embodied AI research." - PLAIcraft Frequently Asked Questions. They have tasked us with helping them out by answering a few questions for them based on two anonymised datasets regarding player data, and individual play session data. This leads to the question which we attempted to answer for them!

### Our Question

**Can we predict the number of future sessions a player will engage in based on their initial session characteristics and player information?**

To simplify that down more, what our goal was for this data analysis was to utilize a players specific data, as well as the session data for only their very first session to determine if they will return for subsequent sessions, and if they do, how many? 

### The Dataset

Now for the bread and butter, well perhaps the dataset is the bread and the data analysis that comes later is the butter, who knows. Through this brief section we will load the data and talk about it and most importantly help you to understand what all of it means before we get into the next section which is where the real fun begins.

In [2]:
players <- read_csv("data/players.csv")
sessions <- read_csv("data/sessions.csv")

Rows: 196 Columns: 9
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (4): experience, hashedEmail, name, gender
dbl (2): played_hours, age
lgl (3): subscribe, individualId, organizationName

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 1535 Columns: 5
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (3): hashedEmail, start_time, end_time
dbl (2): original_start_time, original_end_time

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [3]:
head(players)

experience,subscribe,hashedEmail,played_hours,name,gender,age,individualId,organizationName
<chr>,<lgl>,<chr>,<dbl>,<chr>,<chr>,<dbl>,<lgl>,<lgl>
Pro,TRUE,f6daba428a5e19a3d47574858c13550499be23603422e6a0ee9728f8b53e192d,30.3,Morgan,Male,9,NA,NA
Veteran,TRUE,f3c813577c458ba0dfef80996f8f32c93b6e8af1fa939732842f2312358a88e9,3.8,Christian,Male,17,NA,NA
Veteran,FALSE,b674dd7ee0d24096d1c019615ce4d12b20fcbff12d79d3c5a9d2118eb7ccbb28,0.0,Blake,Male,17,NA,NA
Amateur,TRUE,23fe711e0e3b77f1da7aa221ab1192afe21648d47d2b4fa7a5a659ff443a0eb5,0.7,Flora,Female,21,NA,NA
Regular,TRUE,7dc01f10bf20671ecfccdac23812b1b415acd42c2147cb0af4d48fcce2420f3e,0.1,Kylie,Male,21,NA,NA
Amateur,TRUE,f58aad5996a435f16b0284a3b267f973f9af99e7a89bee0430055a44fa92f977,0.0,Adrian,Female,17,NA,NA


Alright, so what we've done is loaded both datasets and then directly above this is a sneak-peek at the player information dataset! A lot of the data here is pretty obvious such as **name**, **gender**, and **age**. The other columns of data represent somewhat less obvious data such as the **experience** column denoting the players self-identified minecraft skill level which they can choose as Pro, Amateur, Regular, and more. To the right of that is the **subscribe** which shows whether or not they enrolled in the email list of the study. Look right once more and we see the **hashedEmail** column which is an anonymised encoding of each users email (this comes in handy later on), and finally is the **played_hours** data which represents each users *total* hours played on the server. As we are not cleaning up the data yet, the individualId and organizationName columns are still visible but they contain no data so when we move on later we will take care of that.

In [4]:
head(sessions)

hashedEmail,start_time,end_time,original_start_time,original_end_time
<chr>,<chr>,<chr>,<dbl>,<dbl>
bfce39c89d6549f2bb94d8064d3ce69dc3d7e72b38f431d8aa0c4bf95ccee6bf,30/06/2024 18:12,30/06/2024 18:24,1.71977e+12,1.71977e+12
36d9cbb4c6bc0c1a6911436d2da0d09ec625e43e6552f575d4acc9cf487c4686,17/06/2024 23:33,17/06/2024 23:46,1.71867e+12,1.71867e+12
f8f5477f5a2e53616ae37421b1c660b971192bd8ff77e3398304c7ae42581fdc,25/07/2024 17:34,25/07/2024 17:57,1.72193e+12,1.72193e+12
bfce39c89d6549f2bb94d8064d3ce69dc3d7e72b38f431d8aa0c4bf95ccee6bf,25/07/2024 03:22,25/07/2024 03:58,1.72188e+12,1.72188e+12
36d9cbb4c6bc0c1a6911436d2da0d09ec625e43e6552f575d4acc9cf487c4686,25/05/2024 16:01,25/05/2024 16:12,1.71665e+12,1.71665e+12
bfce39c89d6549f2bb94d8064d3ce69dc3d7e72b38f431d8aa0c4bf95ccee6bf,23/06/2024 15:08,23/06/2024 17:10,1.71916e+12,1.71916e+12


Now we're looking at the combined players sessions dataset which includes session data for the sessions of all the players in the study. Again, this is only a sneak-peek into it, when in reality it has over three thousand rows. We still see **hashedEmail** in this dataset, but the rest of the variables are different! The **start_time** and **end_time** columns represent the corresponding date and time in which each play-session was initiated and terminated. The **original_start_time** and **original_end_time** columns are a bit more complicated, but they are just the UNIX timestamp versions of the other start and end time columns, so don't worry about them!

---

## Methods and Results

### Data Wrangling

As mentioned in the introductory section, we noted some issues with the current data such as the **individualId** and **organizationName** columns not being populated with any data, and the **original_start_time** and **original_end_time** columns being rendered redundant by the existence of the much easier to work with **start_time** and **end_time** columns.

To fix these and make our data *tidy*, we will modify the current dataframes to do so by selecting which columns we would like to exclude by using subset and select! - starting with the players **players** dataframe:

In [5]:
players <- subset(players, select = -c(individualId, organizationName))
head(players)

experience,subscribe,hashedEmail,played_hours,name,gender,age
<chr>,<lgl>,<chr>,<dbl>,<chr>,<chr>,<dbl>
Pro,TRUE,f6daba428a5e19a3d47574858c13550499be23603422e6a0ee9728f8b53e192d,30.3,Morgan,Male,9
Veteran,TRUE,f3c813577c458ba0dfef80996f8f32c93b6e8af1fa939732842f2312358a88e9,3.8,Christian,Male,17
Veteran,FALSE,b674dd7ee0d24096d1c019615ce4d12b20fcbff12d79d3c5a9d2118eb7ccbb28,0.0,Blake,Male,17
Amateur,TRUE,23fe711e0e3b77f1da7aa221ab1192afe21648d47d2b4fa7a5a659ff443a0eb5,0.7,Flora,Female,21
Regular,TRUE,7dc01f10bf20671ecfccdac23812b1b415acd42c2147cb0af4d48fcce2420f3e,0.1,Kylie,Male,21
Amateur,TRUE,f58aad5996a435f16b0284a3b267f973f9af99e7a89bee0430055a44fa92f977,0.0,Adrian,Female,17


That's looking much tidier already! Let's do the **sessions** dataframe next:

In [6]:
sessions <- subset(sessions, select = -c(original_start_time, original_end_time))
head(sessions)

hashedEmail,start_time,end_time
<chr>,<chr>,<chr>
bfce39c89d6549f2bb94d8064d3ce69dc3d7e72b38f431d8aa0c4bf95ccee6bf,30/06/2024 18:12,30/06/2024 18:24
36d9cbb4c6bc0c1a6911436d2da0d09ec625e43e6552f575d4acc9cf487c4686,17/06/2024 23:33,17/06/2024 23:46
f8f5477f5a2e53616ae37421b1c660b971192bd8ff77e3398304c7ae42581fdc,25/07/2024 17:34,25/07/2024 17:57
bfce39c89d6549f2bb94d8064d3ce69dc3d7e72b38f431d8aa0c4bf95ccee6bf,25/07/2024 03:22,25/07/2024 03:58
36d9cbb4c6bc0c1a6911436d2da0d09ec625e43e6552f575d4acc9cf487c4686,25/05/2024 16:01,25/05/2024 16:12
bfce39c89d6549f2bb94d8064d3ce69dc3d7e72b38f431d8aa0c4bf95ccee6bf,23/06/2024 15:08,23/06/2024 17:10


There's one sneaky thing that may trip us up later if not fixed. We can see that the **start_time** and **end_time** columns are of type *chr (character)* which may make working with them a bit tedious. To resolve this we will employ the **asPOSIXct** function from the base R library to convert them to the type *dttm (date time month)* which will make them far easier to use.

In [7]:
sessions <- mutate(sessions, start_time = as.POSIXct(start_time, format="%d/%m/%Y %H:%M", tz="PST"),
         end_time = as.POSIXct(end_time, format="%d/%m/%Y %H:%M", tz="PST"))

head(sessions)

hashedEmail,start_time,end_time
<chr>,<dttm>,<dttm>
bfce39c89d6549f2bb94d8064d3ce69dc3d7e72b38f431d8aa0c4bf95ccee6bf,2024-06-30 18:12:00,2024-06-30 18:24:00
36d9cbb4c6bc0c1a6911436d2da0d09ec625e43e6552f575d4acc9cf487c4686,2024-06-17 23:33:00,2024-06-17 23:46:00
f8f5477f5a2e53616ae37421b1c660b971192bd8ff77e3398304c7ae42581fdc,2024-07-25 17:34:00,2024-07-25 17:57:00
bfce39c89d6549f2bb94d8064d3ce69dc3d7e72b38f431d8aa0c4bf95ccee6bf,2024-07-25 03:22:00,2024-07-25 03:58:00
36d9cbb4c6bc0c1a6911436d2da0d09ec625e43e6552f575d4acc9cf487c4686,2024-05-25 16:01:00,2024-05-25 16:12:00
bfce39c89d6549f2bb94d8064d3ce69dc3d7e72b38f431d8aa0c4bf95ccee6bf,2024-06-23 15:08:00,2024-06-23 17:10:00


Now we can use those columns to do something that will be very important to our analysis which is computing the duration of the sessions (difference between start_time and end_time in minutes)!

In [8]:
sessions <- mutate(sessions, session_duration = as.numeric(difftime(end_time, start_time, units = "mins")))

head(sessions)

hashedEmail,start_time,end_time,session_duration
<chr>,<dttm>,<dttm>,<dbl>
bfce39c89d6549f2bb94d8064d3ce69dc3d7e72b38f431d8aa0c4bf95ccee6bf,2024-06-30 18:12:00,2024-06-30 18:24:00,12
36d9cbb4c6bc0c1a6911436d2da0d09ec625e43e6552f575d4acc9cf487c4686,2024-06-17 23:33:00,2024-06-17 23:46:00,13
f8f5477f5a2e53616ae37421b1c660b971192bd8ff77e3398304c7ae42581fdc,2024-07-25 17:34:00,2024-07-25 17:57:00,23
bfce39c89d6549f2bb94d8064d3ce69dc3d7e72b38f431d8aa0c4bf95ccee6bf,2024-07-25 03:22:00,2024-07-25 03:58:00,36
36d9cbb4c6bc0c1a6911436d2da0d09ec625e43e6552f575d4acc9cf487c4686,2024-05-25 16:01:00,2024-05-25 16:12:00,11
bfce39c89d6549f2bb94d8064d3ce69dc3d7e72b38f431d8aa0c4bf95ccee6bf,2024-06-23 15:08:00,2024-06-23 17:10:00,122


Both of our dataframes look very tidy now, although working with two separate dataframes may overcomplicate our analysis and code both for us to work with and for you to understand. While we are doing that, we will also manipulate the data to introduce two new columns of further useful metrics regarding our analysis. Since we're basing our predictions off of every players first session, we really want to know their first session length: **first_session_duration** and how many sessions they return to play after that one: **future_sessions_played**.

In [12]:
player_session_combined <- players |> inner_join(sessions, by = "hashedEmail")

player_session_finalized <- player_session_combined |>
  group_by(hashedEmail) |>
  mutate(first_session_duration = first(session_duration),
         future_sessions_count = n() - 1) |>
  ungroup()

head(player_session_finalized)

experience,subscribe,hashedEmail,played_hours,name,gender,age,start_time,end_time,session_duration,first_session_duration,future_sessions_count
<chr>,<lgl>,<chr>,<dbl>,<chr>,<chr>,<dbl>,<dttm>,<dttm>,<dbl>,<dbl>,<dbl>
Pro,TRUE,f6daba428a5e19a3d47574858c13550499be23603422e6a0ee9728f8b53e192d,30.3,Morgan,Male,9,2024-08-08 00:21:00,2024-08-08 01:35:00,74,74,26
Pro,TRUE,f6daba428a5e19a3d47574858c13550499be23603422e6a0ee9728f8b53e192d,30.3,Morgan,Male,9,2024-09-09 22:30:00,2024-09-09 22:37:00,7,74,26
Pro,TRUE,f6daba428a5e19a3d47574858c13550499be23603422e6a0ee9728f8b53e192d,30.3,Morgan,Male,9,2024-08-08 02:41:00,2024-08-08 03:25:00,44,74,26
Pro,TRUE,f6daba428a5e19a3d47574858c13550499be23603422e6a0ee9728f8b53e192d,30.3,Morgan,Male,9,2024-09-10 15:07:00,2024-09-10 15:29:00,22,74,26
Pro,TRUE,f6daba428a5e19a3d47574858c13550499be23603422e6a0ee9728f8b53e192d,30.3,Morgan,Male,9,2024-05-05 22:21:00,2024-05-05 23:17:00,56,74,26
Pro,TRUE,f6daba428a5e19a3d47574858c13550499be23603422e6a0ee9728f8b53e192d,30.3,Morgan,Male,9,2024-04-06 22:24:00,2024-04-06 23:33:00,69,74,26


experience,subscribe,hashedEmail,played_hours,name,gender,age,start_time,end_time,session_duration,first_session_duration,future_sessions_count
<chr>,<lgl>,<chr>,<dbl>,<chr>,<chr>,<dbl>,<dttm>,<dttm>,<dbl>,<dbl>,<dbl>
Amateur,FALSE,f19e136ddde68f365afc860c725ccff54307dedd13968e896a9f890c40aea436,2.3,Harlow,Male,17,2024-09-08 20:04:00,2024-09-08 22:02:00,118,7,5
Amateur,FALSE,f19e136ddde68f365afc860c725ccff54307dedd13968e896a9f890c40aea436,2.3,Harlow,Male,17,2024-08-23 21:59:00,2024-08-23 22:06:00,7,7,5
Amateur,FALSE,f19e136ddde68f365afc860c725ccff54307dedd13968e896a9f890c40aea436,2.3,Harlow,Male,17,2024-09-09 02:17:00,2024-09-09 02:45:00,28,7,5
Amateur,FALSE,f19e136ddde68f365afc860c725ccff54307dedd13968e896a9f890c40aea436,2.3,Harlow,Male,17,2024-08-23 21:39:00,2024-08-23 21:53:00,14,7,5
Amateur,FALSE,f19e136ddde68f365afc860c725ccff54307dedd13968e896a9f890c40aea436,2.3,Harlow,Male,17,2024-09-08 19:40:00,2024-09-08 19:45:00,5,7,5
Pro,TRUE,d9473710057f7d42f36570f0be83817a4eea614029ff90cf50d8889cdd729d11,0.2,Ahmed,Other,91,2024-05-08 19:29:00,2024-05-08 19:44:00,15,15,0


One crucial thing we must do prior to moving on is to remove every row for each hashedEmail **except** the session which occurs, and while we do that we may as well remove all the other columns which will be not applicable to our analysis such as the **start_time** and **end_time** columns, as well as individual session durations, players names, and their hashed emails. This is to cut any fluff that would mess with our analysis as only certain variables will be used as predictors!

In [20]:
final_wrangled_data <- player_session_finalized |>
  group_by(hashedEmail) |>
  filter(start_time == min(start_time)) |> # Keep only the first session based on earliest start_time
  ungroup() |>
  select(age, gender, subscribe, experience, first_session_duration, future_sessions_count)                     

final_wrangled_data

age,gender,subscribe,experience,first_session_duration,future_sessions_count
<dbl>,<chr>,<lgl>,<chr>,<dbl>,<dbl>
9,Male,TRUE,Pro,74,26
17,Male,TRUE,Veteran,12,2
17,Male,FALSE,Veteran,5,0
21,Female,TRUE,Amateur,50,0
21,Male,TRUE,Regular,9,0
21,Male,FALSE,Amateur,5,0
17,Male,TRUE,Amateur,13,0
23,Female,TRUE,Veteran,18,2
25,Female,TRUE,Beginner,101,0


![](data/wrangled.jpg)

We agree with the cowboy on this one, that data is looking very wrangled and ready to be analyzed, so let's do it!

## D